In [1]:
import tweepy
import re
import pickle
import numpy as np
from tweepy import OAuthHandler
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# Initializing the keys
consumer_key = 'CIFjmXPDQccyG2uLkFrO4LJcx'
consumer_secret = '6U88AaFE2wd7tjvDKPvXEv7qFthxi82yMXmt2V2CPYvLQyY5JS' 
access_token = '1181430333033500672-Qkfih9BiNOBIAqhnaTmapqhbRaKTTL'
access_secret ='24FxebVyhzy1NBtSaET6FTQNDokI87zCQO1oxwO4Nv1OM'

In [3]:
# Initializing the tokens
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
args = ['#MeToo']
api = tweepy.API(auth,timeout=10)

In [4]:
# Fetching the tweets
list_tweets = []

query = args[0]
if len(args) == 1:
    for status in tweepy.Cursor(api.search,q=query+" -filter:retweets",lang='en',result_type='recent').items(100):
        list_tweets.append(status.text)

In [5]:
list_tweets

["@katerbland https://t.co/ps0ktAwrQu was just reading this article and noticed you've spelt the founder of the… https://t.co/y0cyVaKBLa",
 'How #MeToo revealed the central rift within feminism today https://t.co/kpDam3wibS',
 "A year later, what's next for India's #MeToo movement? @nandita_singh1, @khanthefatima, @MandhaniApoorva &amp; I spoke… https://t.co/6ffCF0Kawu",
 'Been trying to do this all day, imagine a train looking out and six bronze Aussie lifeguards are standing on the bl… https://t.co/OhkUJn3YK9',
 'I just had the pleasure of feeling someone’s nipples on my back as I’m sitting at the bar.... lol #metoo? #wtf',
 'INFOGRAPHIC on #Nifty companies with maximum sexual harassment complaints filed under PoSH Act in FY19 - compiled b… https://t.co/36LpHd5sAj',
 '@DFBHarvard @u_funky More whistle blowers just like the parade of #metoo who lined up to testify against Kavanaugh… https://t.co/GCiqFteUmr',
 'I really want another season of @TheRoyalsTV. I understand there were some 

In [6]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, ' ', input_txt)
        
    return input_txt    

In [7]:
# remove twitter handles (@user)
list_tweets = np.vectorize(remove_pattern)(list_tweets, "@[\w]*")
list_tweets = np.vectorize(remove_pattern)(list_tweets, "\s+https://t.co/[\w\d]*$")
list_tweets = np.vectorize(remove_pattern)(list_tweets, "^https://t.co/[a-zA-Z0-9]*\s")
list_tweets = np.vectorize(remove_pattern)(list_tweets, "\s+https://t.co/[a-zA-Z0-9]*\s")
list_tweets = np.vectorize(remove_pattern)(list_tweets, "\d")
list_tweets = np.vectorize(remove_pattern)(list_tweets, "[\n]")

In [8]:
list_tweets

array([" was just reading this article and noticed you've spelt the founder of the… ",
       'How #MeToo revealed the central rift within feminism today ',
       "A year later, what's next for India's #MeToo movement?  ,  ,   &amp; I spoke… ",
       'Been trying to do this all day, imagine a train looking out and six bronze Aussie lifeguards are standing on the bl… ',
       'I just had the pleasure of feeling someone’s nipples on my back as I’m sitting at the bar.... lol #metoo? #wtf',
       'INFOGRAPHIC on #Nifty companies with maximum sexual harassment complaints filed under PoSH Act in FY   - compiled b… ',
       '    More whistle blowers just like the parade of #metoo who lined up to testify against Kavanaugh… ',
       'I really want another season of  . I understand there were some #MeToo issues behind the scenes, that’s… ',
       'Nurse struck off for failing to report child sexual abuse concerns - BBC News #metoo ',
       'The Perfect Alibi (Robin Lockwood # ) [EPUB] #F

In [9]:
# Loading the vectorizer and classfier
with open('classifier.pickle','rb') as f:
    classifier = pickle.load(f)
    
with open('tfidfmodel.pickle','rb') as f:
    tfidf = pickle.load(f)   

In [15]:
# Preprocessing the tweets and predicting sentiment
total_pos,total_neg=0,0
for tweet in list_tweets:
    tweet = tweet.lower()
    tweet = re.sub(r"\W"," ",tweet)
    tweet = re.sub(r"\d"," ",tweet)
    tweet = re.sub(r"\s+[a-z]\s+"," ",tweet)
    tweet = re.sub(r"\s+[a-z]$"," ",tweet)
    tweet = re.sub(r"^[a-z]\s+"," ",tweet)
    tweet = re.sub(r"\s+"," ",tweet)
    tweet = re.sub(r"\s$","",tweet)
    tweet = re.sub(r"^\s","",tweet)
    sent = classifier.predict(tfidf.transform([tweet]).toarray())
    print(tweet," : ",sent)
    if sent[0] == 1:
        total_pos = total_pos+1
    else:
        total_neg = total_neg+1

was just reading this article and noticed you ve spelt the founder of the  :  [1]
how metoo revealed the central rift within feminism today  :  [1]
year later what next for india metoo movement amp spoke  :  [1]
been trying to do this all day imagine train looking out and six bronze aussie lifeguards are standing on the bl  :  [1]
just had the pleasure of feeling someone nipples on my back as m sitting at the bar lol metoo wtf  :  [1]
infographic on nifty companies with maximum sexual harassment complaints filed under posh act in fy compiled  :  [1]
more whistle blowers just like the parade of metoo who lined up to testify against kavanaugh  :  [1]
really want another season of understand there were some metoo issues behind the scenes that  :  [1]
nurse struck off for failing to report child sexual abuse concerns bbc news metoo  :  [1]
the perfect alibi robin lockwood epub full page metoo  :  [1]
comments under this the modern young boy girls think that society creates issue on mix fri

In [13]:
for tweet in range(list_tweets):
    sent = classifier.predict(tfidf.transform(tweet).toarray())
    print(tweet," : ",sent)

TypeError: only integer scalar arrays can be converted to a scalar index

In [16]:
# Visualizing the results
import matplotlib.pyplot as plt
import numpy as np
objects = ['Positive','Negative']
y_pos = np.arange(len(objects))

plt.bar(y_pos,[total_pos,total_neg],alpha=0.5)
plt.xticks(y_pos,objects)
plt.ylabel('Number')
plt.title('Number of Postive and Negative Tweets')
plt.show()

<Figure size 640x480 with 1 Axes>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#Common words used in Nov month tweets, using WordCloud
all_words = ' '.join([text for text in list_tweets])
from wordcloud import WordCloud
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(all_words)

plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
sent[0]